What do we need to achive?

- Data Acquisition -     20
    * Diversity of sources (at least one must be dynamic – full marks for using both APIs and web scraping – penalties will be applied for re-using examples from class)
    * Appropriate use of merging and concatenation.
    
Data Wrangling and EDA  -   30
Data Analysis -  35
Originality and challenge 15

BONUS
Big Data ProcessingTechniques - 5

Your research report must be in a Jupyter Notebook format and thus executable and repeatable.

Clearly introduce your problem domain, articulate your research questions and provide an executive summary at the beginning. Follow the provided Jupyter notebook template.
You must document and explain the reasoning behind the coding steps you are taking and provide explanations of all your graphs and tables as is appropriate. Make sure you label all aspects of your graphs.
The activities listed under the five stages in the workflow diagram above are a guide only. This means that operations like group-by statements as well as pivot tables could be a part of the ‘Data Wrangling’ phase as EDA, and not only a part of the data analysis phase. Finally, please run your report through an external spell checker and feel free to use ChatGPT judiciously to help you as discussed in class.


Question I am trying to answer

Is there money to be made by taking into account if a tennis player has famous parents, when betting on a tennis player.
Is having a famous parent an indicator of future success.

Two things that make me wonder this. Firstly there are two of the top 100 woman tennis players have billionare parent. Second a tweet I saw wondering why so many formula one drivers fathers have hyperlinks in wikipedia (hinting that you need a rich father to become a formular 1 driver).

Info we will need
- Betting odds for tennis matches from the last 10 years
- List of tennis players from the last 10 years
- Table of past match results
- List of tennis players that have wiki pages, and a boolean if there parents have wiki pages

How else can we work out if a tennis player has a rich parent?



First would be to see if there is a correlation between rich partents and tennis success. 
we could look at win % vs parents with hyperlinks
We could look at 


Then, if the answer is yes, then look to see if this is taken into account in the betting odds.

For each player, we could work out how much you would have won / lost, if you had bet a constant amount on that player, over there last x number of games. We can then have a way of saying who is a profitable player and who is an unprofitable player to bet on. We can then see if this correlates with if they have rich/famous parents.














In [1]:
# this gives a list of ATP tornaments using https://rapidapi.com/sportcontentapi/api/tennis-live-data

import requests
api_key = '8556d4b2f5mshddae5c2b7778158p1b7b83jsn131f0acf695b'
# Replace YOUR_API_KEY with your RapidAPI key
headers = {
    "x-rapidapi-key": "8556d4b2f5mshddae5c2b7778158p1b7b83jsn131f0acf695b",
    "x-rapidapi-host": "tennis-live-data.p.rapidapi.com"
}

# Replace the example endpoint with the actual endpoint you want to call
endpoint = "https://tennis-live-data.p.rapidapi.com/tournaments/ATP/2020"



response = requests.get(endpoint, headers=headers)

# Print the response content
print(response.json())

{'meta': {'title': 'Live Tennis API - Tennis Tournaments for ATP 2020', 'description': 'List of tournaments for given code and season', 'fields': {'id': 'Integer - unique tournament_id for use querying other endpoints', 'name': 'String', 'city': 'String', 'country': 'String', 'surface': 'String - tournament surface type i.e Clay, Hard Indoor, Hard Outdoor, Grass', 'code': 'String - tour_code', 'start_date': 'Date - YYYY-MM-DD', 'end_date': 'Date - YYYY-MM-DD', 'season': 'Integer - tour_season_id'}}, 'results': [{'id': 1143, 'name': 'Qatar ExxonMobil Open', 'city': 'Doha', 'country': 'Qatar', 'surface': 'Outdoor Hard', 'code': 'ATP', 'start_date': '2020-01-06', 'end_date': '2020-01-11', 'season': 2020, 'country_code': 'QAT'}, {'id': 1144, 'name': 'Adelaide International', 'city': 'Adelaide', 'country': 'Australia', 'surface': 'Outdoor Hard', 'code': 'ATP', 'start_date': '2020-01-12', 'end_date': '2020-01-18', 'season': 2020, 'country_code': 'AUS'}, {'id': 1145, 'name': 'ASB Classic', 'c

In [3]:
import requests

url = "https://tennis-live-data.p.rapidapi.com/players/ATP"

headers = {
	"X-RapidAPI-Key": "8556d4b2f5mshddae5c2b7778158p1b7b83jsn131f0acf695b",
	"X-RapidAPI-Host": "tennis-live-data.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response.json())

{'meta': {'title': 'Live Tennis API - Players By Tour: ATP', 'description': 'List of Players in Specified Tour', 'fields': {'id': 'Integer - player_id', 'first_name': 'String', 'last_name': 'String', 'full_name': 'String', 'country': 'String'}}, 'results': {'players': [{'id': 89304, 'first_name': 'Novak', 'last_name': 'Djokovic', 'full_name': 'Novak Djokovic', 'country': 'Serbia'}, {'id': 86928, 'first_name': 'Rafael', 'last_name': 'Nadal', 'full_name': 'Rafael Nadal', 'country': 'Spain'}, {'id': 342990, 'first_name': 'Alexander', 'last_name': 'Zverev', 'full_name': 'Alexander Zverev', 'country': 'Germany'}, {'id': 91668, 'first_name': 'Potro', 'last_name': 'Del', 'full_name': 'Potro Del', 'country': 'Argentina'}, {'id': 93378, 'first_name': 'Kevin', 'last_name': 'Anderson', 'full_name': 'Kevin Anderson', 'country': 'South Africa'}, {'id': 86064, 'first_name': 'Roger', 'last_name': 'Federer', 'full_name': 'Roger Federer', 'country': 'Switzerland'}, {'id': 94410, 'first_name': 'Kei', 'l

In [1]:
import requests
from bs4 import BeautifulSoup

def parents_have_wiki(url):
    parents = []  # initialize the parents list
    last_name = None  # initialize the last name variable
    
    # Make a request to the URL and get the HTML response
    response = requests.get(url)
    html = response.content
    
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find the "Personal life" or "Early life" section on the page
    personal_life_section = soup.find('span', {'id': 'Personal_life'})
    #if personal_life_section
    print("personal life text" + personal_life_section.)
    
    # Look for parents in the infobox
    infobox = soup.find('table', {'class': 'infobox'})
    if infobox:
        for row in infobox.find_all('tr'):
            th = row.find('th')
            if th and 'Parent' in th.text:
                td = row.find('td')
                for link in td.find_all('a'):
                    href = link.get('href')
                    if href and '/wiki/' in href and 'Wikipedia:' not in href:
                        # Add the parent's URL to the list
                        parents.append(href)
                        
                        
    
    # Look for parents in the "Personal life" section
    if personal_life_section is not None:
        personal_life_content = personal_life_section.parent.find_next_sibling('div', {'class': 'hatnote'})
        print("persona life content" + personal_life_content.text)
        if personal_life_content is not None:
            for link in personal_life_content.find_all('a'):
                href = link.get('href')
                print(href)
                if href and '/wiki/' in href and 'Wikipedia:' not in href:
                    # Check if the hyperlink points to a person with the same last name
                    link_text = link.text
                    print(link_text)
                    if last_name is None:
                        # Get the last name of the person whose page we're on
                        name = soup.find('h1', {'id': 'firstHeading'}).text
                        last_name = name.split()[-1]
                        print(last_name)
                    if last_name in link_text:
                        # Add the parent's URL to the list
                        parents.append(href)
    
    # Check if both parents have Wikipedia pages
    for parent in parents:
        parent_url = 'https://en.wikipedia.org' + parent
        parent_response = requests.get(parent_url)
        parent_soup = BeautifulSoup(parent_response.content, 'html.parser')
        parent_title = parent_soup.find('h1', {'id': 'firstHeading'}).text
        if parent_title != 'Wikipedia':
            # Check if the parent's page exists
            return True
    
    return False



In [28]:
parents_have_wiki("https://en.wikipedia.org/wiki/Je%C4%BCena_Ostapenko")

Personal life
persona life contentMain article: Jeļena Ostapenko career statistics
/wiki/Je%C4%BCena_Ostapenko_career_statistics
Jeļena Ostapenko career statistics
Ostapenko


True

In [22]:
parents_have_wiki("https://en.wikipedia.org/wiki/Jessica_Pegula")

Jessica Pegula career statistics
Pegula


True

In [2]:
parents_have_wiki("https://en.wikipedia.org/wiki/Zhang_Zhizhen")

AttributeError: 'NoneType' object has no attribute 'text'